<a href="https://colab.research.google.com/github/eip4-mars/EIP4P2/blob/master/Session2/Initial_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchsummary import summary
from IPython.display import clear_output

use_cuda= torch.cuda.is_available()
device=torch.device('cuda' if use_cuda else 'cpu')

model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, 4)
model = nn.Sequential(model, nn.LogSoftmax())
model = model.to(device)
summary(model, input_size=(3,224,224))

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount =True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!unzip /content/drive/My\ Drive/Session\ 2\ Dataset.zip
clear_output()

In [ ]:
##rm -rf Session2_Dataset/

In [4]:
!du -sh Session\ 2\ Dataset/

1.6G	Session 2 Dataset/


In [5]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm
import traceback
import warnings

warnings.filterwarnings("error")

path = "/content/Session 2 Dataset/"
test_ratio = 0.2


def create_dataset_csv_split(path, test_ratio = 0.2):
    classes = os.listdir(path)
    train = []
    test = []
    labels = pd.DataFrame(classes,index=range(len(classes)), columns=['Label'])
    labels.reset_index(inplace=True)


    #print(labels)
    for i in range(len(classes)):
        child_path = path+classes[i]+'/'
        imgs = os.listdir(child_path)
        #print(len(tr),classes[i])
        trn,tst = train_test_split(imgs, test_size=test_ratio, random_state=42)
        
        train += [[child_path+x, i] for x in trn]
        test += [[child_path+x, i] for x in tst]

    # train1 = train.copy()
    # test1 = test.copy()

    for impath in tqdm(train):
        try:
            im = Image.open(impath[0])
            im.close()
        except:
            train.remove(impath)
            pass

    for impath in tqdm(test):
        try:
            im = Image.open(impath[0])
            im.close()
        except:
            test.remove(impath)
            pass

    train_df = pd.DataFrame(train,columns=['File_name', 'label_index'])
    test_df = pd.DataFrame(test,columns=['File_name', 'label_index'])

    print(train_df.shape, test_df.shape, labels.shape)
    #train_df_1 = pd.concat([train_df[["File_name"]], pd.get_dummies(train_df.label_index, prefix="label")], axis = 1)
    #test_df_1 = pd.concat([test_df[["File_name"]], pd.get_dummies(test_df.label_index, prefix="label")], axis = 1)

    train_df.to_csv('train.csv',index=False)
    test_df.to_csv('test.csv',index=False)
    labels.to_csv('labels.csv',index=False)
    warnings.filterwarnings("ignore")



In [6]:
create_dataset_csv_split(path,test_ratio)

100%|██████████| 4148/4148 [00:00<00:00, 6798.36it/s]


(16582, 2) (4148, 2) (4, 2)


In [16]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
import random

class custDataset(Dataset):
	"""Face Landmarks dataset."""

	def __init__(self, csv_file, transform=None):
		self.frame = pd.read_csv(csv_file, header=0)
		self.transform = transform

	def __getitem__(self, idx):
		image_name = self.frame.iloc[idx, 0]
		label = self.frame.iloc[idx, 1]
		
		image = Image.open(image_name).convert('RGB')
		if self.transform:
			image = self.transform(image)
		#sample = {'image': image, 'label': label}
		return image, label
		
	def __len__(self):
		return len(self.frame)

def getTrainData(batch_size=64):
	
	stats = {'mean': [0.3931, 0.3785, 0.3606],
			 'std': [0.1965, 0.1813, 0.1779]}

	transformed_training = custDataset(csv_file='/content/train.csv',
											transform=transforms.Compose([
											transforms.Resize(224),
											transforms.CenterCrop(224),
											transforms.RandomHorizontalFlip(),
											transforms.RandomRotation(10),
											transforms.ToTensor(),
											transforms.Normalize(stats['mean'],stats['std'])
										]))

	dataloader_training = DataLoader(transformed_training, batch_size,
									 shuffle=True, num_workers=4, pin_memory=False)

	return dataloader_training



def getTestData(batch_size=64):

	stats = {'mean': [0.3931, 0.3785, 0.3606],
			 'std': [0.1965, 0.1813, 0.1779]}
	
	transformed_testing = custDataset(csv_file='/content/test.csv',
										transform=transforms.Compose([
										transforms.Resize(224),
										transforms.CenterCrop(224),
										transforms.ToTensor(),
										transforms.Normalize(stats['mean'],stats['std'])
									]))

	dataloader_testing = DataLoader(transformed_testing, batch_size,
									shuffle=True, num_workers=4, pin_memory=False)

	return dataloader_testing


In [17]:
train_loader = getTrainData(64)
test_loader = getTestData(64)

In [9]:
!git clone https://github.com/eip4-mars/eva4_lib.git

Cloning into 'eva4_lib'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 22 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [11]:
from eva4_lib.utils.train_test import train , test, predict
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = optim.SGD(model.parameters(), lr=0.01,  momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=1e-06, eps=1e-06)

for epoch in range(1, 3):
    curr_lr=optimizer.param_groups[0]['lr']
    print(f'Epoch: {epoch} Learning_Rate {curr_lr}')
    train(model, device, train_loader, optimizer, epoch)
    test_acc1 = test(model, device, test_loader)
    print('Test acc:', test_acc1)
    scheduler.step(test_acc1)

  0%|          | 0/260 [00:00<?, ?it/s]

Epoch: 1 Learning_Rate 0.01


Loss=0.2260768860578537 Batch_id=259 Accuracy=87.63: 100%|██████████| 260/260 [04:21<00:00,  1.00s/it]
  0%|          | 0/260 [00:00<?, ?it/s]


Test set: Average loss: 0.2981, Accuracy: 3680/4148 (88.72%)

Test acc: 88.71745419479267
Epoch: 2 Learning_Rate 0.01


Loss=0.31160905957221985 Batch_id=259 Accuracy=88.80: 100%|██████████| 260/260 [04:11<00:00,  1.03it/s]



Test set: Average loss: 0.3013, Accuracy: 3697/4148 (89.13%)

Test acc: 89.12729026036644


In [35]:
fail=[]
for eval_data, eval_target in test_loader:
  eval_data, eval_target = eval_data.to(device), eval_target.to(device)
  eval_out = model(eval_data)
  target_lbl=eval_target.cpu().numpy()
  pred_lbl=eval_out.argmax(1).cpu().numpy()
  #print(pred_lbl)
  #print(eval_target)
  for i in range(64):
    if target_lbl[i] != pred_lbl[i]:fail.append([eval_data[i].cpu(),target_lbl[i],pred_lbl[i],eval_out[i,target_lbl[i]].cpu(),eval_out()[i,pred_lbl[i]].cpu()])
  
  print('fail_count : '+str(len(fail)))
  if len(fail)>100: break

print('Returned Miscalssified images # : {}'.format(len(fail)))
print('** Note : If count of returned image is less than {}, then there arent enough misclassified images in the loader passed'.format(100))


  # fig = plt.figure(figsize=(16,16))
  # cnt=0
  # for i in fail[:25]:
  #   ax=fig.add_subplot(5, 5, cnt+1)
  #   img=np.squeeze(eval_data[i].cpu().numpy()[0])
  #   ax.imshow(img)
  #   cnt+=1
  #   ax.set_title("Actual : "+str(target_lbl[i])+"  Predicted : "+str(pred_lbl[i]))
  # plt.savefig('L1_Misclassified.png')
  # plt.show()

fail_count : 6
fail_count : 10
fail_count : 17


RuntimeError: ignored

In [34]:
import gc
gc.collect()

2511

In [36]:
fail

[[tensor([[[2.1107, 2.0907, 2.0907,  ..., 2.1506, 2.1705, 2.1705],
           [2.1107, 2.0907, 2.0907,  ..., 2.1506, 2.1705, 2.1705],
           [2.1107, 2.0907, 2.0907,  ..., 2.1506, 2.1705, 2.1705],
           ...,
           [1.7913, 1.7913, 1.7913,  ..., 1.8313, 1.8512, 1.8512],
           [1.7913, 1.7913, 1.7913,  ..., 1.8313, 1.8512, 1.8512],
           [1.7913, 1.7913, 1.7913,  ..., 1.8313, 1.8512, 1.8512]],
  
          [[2.3681, 2.3465, 2.3465,  ..., 2.4547, 2.4763, 2.4763],
           [2.3681, 2.3465, 2.3465,  ..., 2.4547, 2.4763, 2.4763],
           [2.3681, 2.3465, 2.3465,  ..., 2.4547, 2.4763, 2.4763],
           ...,
           [2.0004, 2.0004, 2.0004,  ..., 2.0653, 2.0869, 2.0869],
           [2.0004, 2.0004, 2.0004,  ..., 2.0653, 2.0869, 2.0869],
           [2.0004, 2.0004, 2.0004,  ..., 2.0653, 2.0869, 2.0869]],
  
          [[3.2635, 3.2415, 3.2415,  ..., 3.4398, 3.4619, 3.4619],
           [3.2635, 3.2415, 3.2415,  ..., 3.4398, 3.4619, 3.4619],
           [3.2635, 3.

In [16]:
model.to(torch.device('cpu'))
model.eval()
traced_model = torch.jit.trace(model,torch.randn(1,3,244,244))
traced_model.save("Sess2_mobilenetv2_4.pt")

In [ ]:
torch.cuda.empty_cache()